In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

####################
####### RAG 챗봇 구축
###################

# 1. LLM 모델 불러오기
llm = ChatOpenAI(model="gpt-4o-mini")


In [3]:

# 2. 문서 불러오기
import json

from langchain.schema import Document

# JSON 파일 로드 함수
def load_json_as_documents(file_path):
    """
    JSON 파일을 읽어 LangChain의 Document 객체 리스트로 변환
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # JSON 데이터 -> LangChain Document로 변환
    documents = []
    for item in data:
        # 각 항목을 Document 객체로 변환
        content = json.dumps(item, ensure_ascii=False)  # JSON 데이터를 문자열로 직렬화
        documents.append(Document(page_content=content))
    
    return documents

# JSON 파일 경로
file_path = "/Users/t2023-m0072/Desktop/AI_8_CH-3_LLM-RAG_AI_Utilizatioon_App/personal_work/차아인/food_recipes.json"

# JSON 데이터를 문서로 로드
docs = load_json_as_documents(file_path)

# 문서 확인
print(f"Loaded {len(docs)} documents.")
for doc in docs[:3]:  # 샘플 출력
    print(doc)


Loaded 10 documents.
page_content='{"요리명": "미역국", "요리재료": [{"재료": "미역(마른것)", "용량": "5줌(20g)"}, {"재료": "쇠고기(양지머리)", "용량": "120g"}, {"재료": "물", "용량": "8컵(1"}, {"재료": "재래간장 1과", "용량": "1/2큰술(22ml)"}, {"재료": "마늘(다진 마늘)", "용량": "1큰술(10g)"}, {"재료": "소금", "용량": "작은술(3g)"}, {"재료": "참기름", "용량": "작은술(5ml)"}], "기본정보": {"조리시간": "30분", "분량": "4인분 기준", "칼로리": "81kcal (1인분)"}, "조리순서": ["01. 마른 미역은 찬물에 담가 10분간 불린다. 찬물에 바락바락 씻어 거품이 나오지 않을 때까지 헹군다.", "02. 물기를 꼭 짠 후 적당한 크기로 자른 후 재래간장 1/2큰술을 넣고 조물조물 무친다.", "03. 쇠고기는 한입 크기로 썬 후 달군 냄비에 참기름을 두르고 쇠고기, 마늘을 넣어 볶다가 쇠고기가 거의 익으면 미역을 넣고 볶는다.", "04. 03에 물을 넣고 한소끔 끓인다. 재래간장과 소금으로 간하고 더 끓인다. (물 대신 쌀뜨물을 넣으면 더욱 구수하고 맛있는 미역국을 만들 수 있다.)"]}'
page_content='{"요리명": "늙은호박죽", "요리재료": [{"재료": "호박(늙은호박)", "용량": "300g"}, {"재료": "물", "용량": "3컵(600ml)"}, {"재료": "찹쌀가루", "용량": "3큰술(30g)"}, {"재료": "소금", "용량": "1/2작은술(2g)"}, {"재료": "설탕 1큰술(10g)· 재료설명 : 호박 삶을 물(물", "용량": "2컵(400ml))"}, {"재료": "찹쌀가루 물(물 1컵(200ml))· 대체재료 : 찹쌀가루 → 멥쌀가루 (찹쌀가루 대신 멥쌀가루로 대체하여도", "용량": "된다.)"}], "기본정보": {"조리시간"

In [4]:

# 3. 문서 chunking 하기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 4. 자른 chunk들을 embedding 하기
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 5. vector store 구축하기
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 6. retriever 구축하기
retriever = vectorstore.as_retriever()


In [5]:
# 7. 프롬프트 템플릿 구축하기
prompt = ChatPromptTemplate.from_template("""
너는 사용자가 입력한 [재료]에 따라 관련 요리를 알려주는 요리사야.
친절하고 전문적인 말투로 응답해.
이 재료들로 만들 수 있는 요리의 이름들을 알려주고 사용자가 원하는 요리의 상세한 레시피를 아래와 같은 형식으로 알려줘.
- 요리의 이름
- 필요한 재료
- 요리 방법
- 조리 시간
- 인분 수


{context}
질문:
{question}
""")


# 8. 1~7. 요소들을 chain으로 조합하여 RAG 구축 완료
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


####################
####### 구축한 RAG 챗봇 실행
###################

print(rag_chain.invoke("미역으로 만들 수 있는 요리가 있을까요?"))


물론입니다! 미역으로 만들 수 있는 요리 중 하나는 "미역국"입니다. 아래에 미역국의 레시피를 자세히 안내해 드리겠습니다.

### 요리명: 미역국

#### 필요한 재료
- 미역(마른것): 5줌 (20g)
- 쇠고기(양지머리): 120g
- 물: 8컵 (1.9L)
- 재래간장: 1/2큰술 (22ml)
- 마늘(다진 마늘): 1큰술 (10g)
- 소금: 작은술 (3g)
- 참기름: 작은술 (5ml)

#### 요리 방법
1. 마른 미역은 찬물에 담가 10분간 불립니다. 이후 찬물에 바락바락 씻어 거품이 나오지 않을 때까지 헹궈 줍니다.
2. 물기를 꼭 짠 후 적당한 크기로 자르며, 재래간장 1/2큰술을 넣고 조물조물 무칩니다.
3. 쇠고기는 한입 크기로 썬 후, 달군 냄비에 참기름을 두르고 쇠고기와 다진 마늘을 넣어 볶습니다. 쇠고기가 거의 익으면 미역을 넣고 함께 볶습니다.
4. 3번에서 물을 넣고 한소끔 끓인 후, 재래간장과 소금으로 간하며 더 끓입니다. (물 대신 쌀뜨물을 사용하면 더욱 구수하고 맛있는 미역국이 됩니다.)

#### 조리 시간
- 약 30분

#### 인분 수
- 4인분 기준

이렇게 간단하게 미역국을 만들 수 있습니다. 미역국은 담백하면서도 영양가가 풍부하여 많은 사랑을 받는 한국 전통 음식입니다. 즐거운 요리 시간 되세요!


## Done
- RAG 간단하게 구성하고 임시 데이터셋과 연결
- 간단한 프롬프트 작성

## TO DO
- 이 챗봇에 넣을 기능들 생각해보기